## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
# added na filter as Namibia is expressed as NA
city_data_df = pd.read_csv("WeatherPy_database.csv", na_filter = False)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,60.28,67,54,14.97,broken clouds
1,1,Bluff,NZ,-46.6000,168.3333,47.80,63,42,26.82,scattered clouds
2,2,Port Elizabeth,ZA,-33.9180,25.5701,65.10,67,0,24.16,clear sky
3,3,Altamira,BR,-3.2033,-52.2064,93.29,55,75,8.05,broken clouds
4,4,Hermanus,ZA,-34.4187,19.2345,60.80,84,36,14.81,scattered clouds


In [70]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [71]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Georgetown,MY,5.4112,100.3354,81.90,81,20,0.00,few clouds
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,72.01,66,44,9.13,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,74.37,69,17,22.06,few clouds
9,9,Karambakkudi,IN,10.4667,79.1333,75.60,75,99,7.58,overcast clouds
12,12,Albany,US,42.6001,-73.9662,80.64,72,98,13.58,overcast clouds


In [72]:
# 4a. Determine if there are any empty rows.
preferred_cities_df[preferred_cities_df['Country'].isnull()]


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description


In [73]:
preferred_cities_df.count()

City_ID                303
City                   303
Country                303
Lat                    303
Lng                    303
Max Temp               303
Humidity               303
Cloudiness             303
Wind Speed             303
Weather Description    303
dtype: int64

In [74]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#no rows with NA, so commented out dropna
#preferred_cities_df.dropna()
clean_df = preferred_cities_df.copy()

In [75]:
preferred_cities_df.count()

City_ID                303
City                   303
Country                303
Lat                    303
Lng                    303
Max Temp               303
Humidity               303
Cloudiness             303
Wind Speed             303
Weather Description    303
dtype: int64

In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Georgetown,MY,81.90,few clouds,5.4112,100.3354,
2,Vila Franca Do Campo,PT,72.01,scattered clouds,37.7167,-25.4333,
5,Rikitea,PF,74.37,few clouds,-23.1203,-134.9692,
9,Karambakkudi,IN,75.60,overcast clouds,10.4667,79.1333,
12,Albany,US,80.64,overcast clouds,42.6001,-73.9662,
14,Nara,JP,72.68,overcast clouds,34.6851,135.8049,
15,Pierre,US,74.32,clear sky,44.3683,-100.3510,
23,Nouakchott,MR,84.13,overcast clouds,18.0858,-15.9785,
24,Terrasini,IT,75.22,clear sky,38.1460,13.0844,
26,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,


In [77]:
hotel_df.rename(columns={"Weather Description": "Current Description"})

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,81.90,few clouds,5.4112,100.3354,
2,Vila Franca Do Campo,PT,72.01,scattered clouds,37.7167,-25.4333,
5,Rikitea,PF,74.37,few clouds,-23.1203,-134.9692,
9,Karambakkudi,IN,75.60,overcast clouds,10.4667,79.1333,
12,Albany,US,80.64,overcast clouds,42.6001,-73.9662,
...,...,...,...,...,...,...,...
668,San Joaquin,US,88.68,clear sky,37.9333,-121.3011,
669,Shakawe,BW,76.80,scattered clouds,-18.3667,21.8500,
673,Santa Rosa,PH,81.10,overcast clouds,14.3122,121.1114,
676,Ottawa,CA,70.99,moderate rain,45.4112,-75.6981,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping. index: {index}")
        

Hotel not found... skipping. index: 12
Hotel not found... skipping. index: 61
Hotel not found... skipping. index: 76
Hotel not found... skipping. index: 78
Hotel not found... skipping. index: 111
Hotel not found... skipping. index: 155
Hotel not found... skipping. index: 230
Hotel not found... skipping. index: 336
Hotel not found... skipping. index: 418
Hotel not found... skipping. index: 421
Hotel not found... skipping. index: 424
Hotel not found... skipping. index: 425
Hotel not found... skipping. index: 433
Hotel not found... skipping. index: 510
Hotel not found... skipping. index: 513
Hotel not found... skipping. index: 521
Hotel not found... skipping. index: 571
Hotel not found... skipping. index: 599
Hotel not found... skipping. index: 605
Hotel not found... skipping. index: 614


In [79]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Georgetown,MY,81.90,few clouds,5.4112,100.3354,Cititel Penang
2,Vila Franca Do Campo,PT,72.01,scattered clouds,37.7167,-25.4333,Pestana Bahia Praia
5,Rikitea,PF,74.37,few clouds,-23.1203,-134.9692,People ThankYou
9,Karambakkudi,IN,75.60,overcast clouds,10.4667,79.1333,Mathiyazhagan house
12,Albany,US,80.64,overcast clouds,42.6001,-73.9662,NaN
14,Nara,JP,72.68,overcast clouds,34.6851,135.8049,Guesthouse Nara Komachi
15,Pierre,US,74.32,clear sky,44.3683,-100.3510,Ramkota Hotel
23,Nouakchott,MR,84.13,overcast clouds,18.0858,-15.9785,HOTEL IMAN فندق إيمان
24,Terrasini,IT,75.22,clear sky,38.1460,13.0844,Florio Park Hotel
26,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [80]:
hotel_df.count()

City                   303
Country                303
Max Temp               303
Weather Description    303
Lat                    303
Lng                    303
Hotel Name             283
dtype: int64

In [81]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Georgetown,MY,81.90,few clouds,5.4112,100.3354,Cititel Penang
2,Vila Franca Do Campo,PT,72.01,scattered clouds,37.7167,-25.4333,Pestana Bahia Praia
5,Rikitea,PF,74.37,few clouds,-23.1203,-134.9692,People ThankYou
9,Karambakkudi,IN,75.60,overcast clouds,10.4667,79.1333,Mathiyazhagan house
14,Nara,JP,72.68,overcast clouds,34.6851,135.8049,Guesthouse Nara Komachi
15,Pierre,US,74.32,clear sky,44.3683,-100.3510,Ramkota Hotel
23,Nouakchott,MR,84.13,overcast clouds,18.0858,-15.9785,HOTEL IMAN فندق إيمان
24,Terrasini,IT,75.22,clear sky,38.1460,13.0844,Florio Park Hotel
26,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
28,Atuona,PF,77.95,clear sky,-9.8000,-139.0333,Villa Enata


In [82]:
hotel_df.count()

City                   283
Country                283
Max Temp               283
Weather Description    283
Lat                    283
Lng                    283
Hotel Name             283
dtype: int64

In [83]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [84]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [85]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))